### Milestone 2 project Group 13
##### by Finn Andersen, Michael Kennecke and Fabian Schick

In [1]:
#some imports:

In [2]:
import pandas as pd
import spacy
from spacy.lang.en import English
nlp = English()
nlp = spacy.load('en_core_web_md')

In [3]:
#inital prediction

In [4]:
%run -i "classifier.py"

Here is the accuracy for domain: 93.5 %
Here is the accuracy for polarity: 84.5 %


In [5]:
'''
Function to load an comma-separeted file (with test data) into a pandas dataframe

def load_data(test_file):
    input:
        test_file: name of the file which shall be loaded
    output:
        test_data: pandas dataframe which contains the file content
'''
def load_data(test_file):
    test_data = pd.read_csv(test_file, sep=",", header=None)
    return test_data

In [6]:
'''
The functions  below are more or less copies of the functions in the give file "classifier.py"
The difference is, that they do not load the file into a dataframe in the first step.
'''
#calculates polarity for a given list of sentences
def calculate_polarity_i(list_of_sentences):
    test_text = list_of_sentences
    model_name = './polarity_classifier.sav'
    prediction = classify(test_text, model_name)
    return prediction

#calculates domain for a given list of sentences
def calculate_domain_i(list_of_sentences):
    test_text = list_of_sentences
    model_name = './domain_classifier.sav'
    prediction = classify(test_text, model_name)
    return prediction

In [7]:
'''
This funtion gets the occurences of words in negative and positive and assigns positive/negative scores for 
adjectives/verbes. This is better than just to substract positive from negative.
List of verbs/adjectives gets usable now. To avoid zero division and not representative values 
we assign some constants sometimes...

calc_score(pos, neg):
    inputs:
        pos: list of occurences of word in positive sentences
        neg: list of occurences of word in negative sentences
    output:
        res_list: returns the score per word
'''
def calc_score(pos, neg):
    res_list = []
    for item_p, item_n in zip(pos, neg):
        if item_n == 0 and item_p>10:
            res_list.append(100)
        elif item_n == 0 and item_p>=5:
            res_list.append(10)
        elif item_n == 0 and item_p<5:
            res_list.append(-1000)
        elif (item_n+item_p)<4:
            res_list.append(-1000)
        else:
            res_list.append(item_p/item_n)
    return res_list

In [8]:
'''
This funtion returns only some words from the text which have the highest impact on the polarity. Thereby 
the accuracy of the polarity classifier shall be increased.

clean_comment(text):
    inputs:
        text: tokenized comment
    output:
        tmp_text: returns all comments which have one of the following pos-tags: 
                    ['ADJ', 'VERB', 'ADV','AUX', 'PART', 'NUM']
'''

def clean_comment(text):
    tmp_text = ''
    for word in text:
        if word.pos_ in ['ADJ', 'VERB', 'ADV','AUX', 'PART', 'NUM']:
            tmp_text += word.text + ' '
    return (tmp_text)

In the first step we load the testdata into the dataframe df_test. Therefore we use the function load_data(test_file). Afterwards we use spacys nlp function to tokenize the comments.

The next step is to split the comments into sentences, to calculate the domain and polarity per sentence and to tokenize the sentences via spacys nlp function.

In [9]:
#loads data
df_original = load_data('./evaluation_examples.csv')
df_test = load_data('./evaluation_examples.csv')
df_only_text =  df_test.copy()

In [10]:
#prepare data with nlp function (get pos-tags and so on)
df_test[0] = [nlp(item) for item in df_test[0]]


In [11]:
#split comment into sentences
list_sentences = []
for comment in df_test[0]:
    for sent in comment.sents:
        list_sentences.append([sent.text])
df_sentences = pd.DataFrame(list_sentences)

In [12]:
# predict polarity and domain per sentence and add to dataframe
df_sentences[1] = calculate_domain_i(df_sentences[0])
df_sentences[2] = calculate_polarity_i(df_sentences[0])

In [13]:
#prepare data with nlp function (get pos-tags and so on)
df_sentences[0] = [nlp(item) for item in df_sentences[0]]

The next step is to get all adjectives and verbs per domain and to count how often ther occur in positive/negative senteces.

In [14]:
# find out which verbs and adjectives are positive / negative and assign them to their domains
list_adj = []
list_verb = []
for sentence, domain, polarity in zip(df_sentences[0], df_sentences[1], df_sentences[2]):
    for i, word in enumerate(sentence):
        if word.pos_=='ADJ':
            if i>0 and sentence[i-1].lemma_ == 'not':
                tmp_not = 'not'
            if polarity == 4:
                list_adj.append([word.lemma_, domain, 0, 1])
            else:
                list_adj.append([word.lemma_,  domain,1, 0])
        
        elif word.pos_=='VERB':
            tmp_not = ''
            if i>0 and sentence[i-1].lemma_ == 'not':
                tmp_not = 'not'
            if polarity == 4:
                list_verb.append([tmp_not + ' ' + word.lemma_, domain, 0, 1])
            else:
                list_verb.append([tmp_not + ' ' + word.lemma_, domain, 1, 0])

In [15]:
'''
some prepartions of this adjective/verb lists
Aggregate values per word/domain + calulate the afore mentioned score
'''
df_adj = pd.DataFrame(list_adj)
df_adj = df_adj.groupby([0,1]).sum()

df_adj = df_adj.reset_index()
df_adj['diff'] = calc_score(df_adj[2], df_adj[3])


df_verb = pd.DataFrame(list_verb)
df_verb = df_verb.groupby([0,1]).sum()

df_verb = df_verb.reset_index()
df_verb['diff'] = calc_score(df_verb[2],df_verb[3])

The next sections define functions which shall change the polarity of a given comment. They are described detailed in the comments in those sections.

In [16]:
'''
adds nots before word and adjectives, if there is no not before. If there is a not before this will be deleted

assign_nots(comment):
    inputs: 
        comment: an original, tokenized comment
    outputs:
        tmp_comment: an processed comment, which shall have the inverted polarity of the original comment
'''

def assign_nots(comment):
    tmp_comment = ''
    #calculate polarity of current comment
    for i, word in enumerate(comment):
        #1: add 'not' before every adjectiv or verb, if it is not there in the original sentence
        if word.pos_ == 'VERB' or word.pos_ == 'ADJ':
            if comment[i-1].lemma_ != 'not':
                tmp_comment += ' not ' + word.text + ' '
            else:
                tmp_comment += word.text + ' '
        #2: Do not use 'nots' from the original sentences
        elif word.lemma_ == 'not':
            continue
        else: 
                tmp_comment += word.text + ' '
    return tmp_comment

In [17]:
'''
returns if a word (ADJ/VERB) has the desired polarity for a given sentence 

word_has_correct_polarity(word, type_of_word, domain, wanted_polarity):
    inputs:
        word: Text of the token
        type_of_word: ADJ/VERB?
        domain: Domain of the comment which contains the word
        wanted_polarity: "Target-polatity"
    outputs:
        returns if the word has the desired polarity, or not
'''
def word_has_correct_polarity(word, type_of_word, domain, wanted_polarity):
    if type_of_word == 'ADJ':
        if wanted_polarity == 3:
            result = list(df_adj[(df_adj[0]==word.lemma_) & (df_adj[1] == domain)]['diff']>5)
            if result == []:
                return False
            else:
                return result[0]
        if wanted_polarity == 4:
            result = list(df_adj[(df_adj[0]==word.lemma_) & (df_adj[1] == domain)]['diff']<0.33)
            if result == []:
                return False
            else:
                return result[0]
    elif type_of_word == 'VERB':
        if wanted_polarity == 4:
            result = list(df_verb[(df_verb[0]==word.lemma_) & (df_verb[1] == domain)]['diff']>5)
            if result == []:
                return False
            else:
                return result[0]
        if wanted_polarity == 4:
            result = list(df_verb[(df_verb[0]==word.lemma_) & (df_verb[1] == domain)]['diff']<0.33)
            if result == []:
                return False
            else:
                return result[0]
            

In [18]:
'''
Fuction might be useless --> To test

Changes positive adjectives and verbes against negative ones vice versa.

Therefore every sentence will be analysed for its polarity and the words will only be changed if the current 
polarity is not equal to the desired one

If there is a not in the sentence this will be deleted

turn_sentences(comment, polarity, domain):
    inputs:
        comment: tokenized original comment
        polarity: predicted polarity (of the comment, preprocessed with clean_comment(text))
        domain: predicted domain
    outputs:
        tmp_comment: pre-processed comment with (hopefully) inverted polatity
'''

def turn_sentences(comment, polarity, domain):
    tmp_comment = ''
    for sent in comment.sents:
        sent_polarity = calculate_polarity_i([sent.text])[0]
        if sent_polarity != polarity:
            if polarity == 3:
                for word in sent:
                    if word.pos_ == 'ADJ' and word_has_correct_polarity(word, 'ADJ', domain, 4):
                        tmp_comment += df_adj.loc[df_adj[(df_adj[1] == domain)]['diff'].idxmax()][0] + ' '
                    elif word.pos_ == 'VERB' and word_has_correct_polarity(word, 'VERB', domain, 4):
                        tmp_comment += df_verb.loc[df_verb[(df_verb[1] == domain)]['diff'].idxmax()][0] + ' '
                    elif word.lemma_ == 'not':
                        continue
                    else: 
                        tmp_comment+= word.text + ' '
            elif polarity == 4:
                for word in sent:
                    if word.pos_ == 'ADJ' and word_has_correct_polarity(word, 'ADJ', domain, 3):
                        tmp_comment += df_adj.loc[df_adj[(df_adj[1] == domain) & (df_adj['diff']>=0)]['diff'].idxmin()][0] + ' '
                    if word.pos_ == 'VERB' and word_has_correct_polarity(word, 'VERB', domain, 3):
                        tmp_comment += df_verb.loc[df_verb[(df_verb[1] == domain) & (df_verb['diff']>=0)]['diff'].idxmin()][0] + ' '
                    elif word.lemma_ == 'not':
                        continue
                    else: 
                        tmp_comment+= word.text + ' '
        else:
            tmp_comment+=(sent.text)
    return tmp_comment

In [19]:
'''
Changes positive adjectives and verbs against negative ones vice versa.

Therefore all verbs/adjectives will be changed to the most common in their domain for the desired polarity

If there is a not in the sentence this will be deleted


turn_sentences2(comment, polarity, domain):
    inputs:
        comment: tokenized original comment
        polarity: predicted polarity (of the comment, preprocessed with clean_comment(text))
        domain: predicted domain
    outputs:
        tmp_comment: pre-processed comment with (hopefully) inverted polarity
'''

def turn_sentences2(comment, polarity, domain):
    tmp_comment = ''
    for sent in comment.sents:
        if polarity == 4:
            for word in sent:
                if word.pos_ == 'ADJ':
                    tmp_comment += df_adj.loc[df_adj[(df_adj[1] == domain)]['diff'].idxmax()][0] + ' '
                elif word.pos_ == 'VERB':
                    tmp_comment += df_verb.loc[df_verb[(df_verb[1] == domain)]['diff'].idxmax()][0] + ' '
                elif word.lemma_ == 'not':
                    continue
                else: 
                    tmp_comment+= word.text + ' '
        elif polarity == 3:
            for word in sent:
                if word.pos_ == 'ADJ':
                    tmp_comment += df_adj.loc[df_adj[(df_adj[1] == domain) & (df_adj['diff']>=0)]['diff'].idxmin()][0] + ' '
                if word.pos_ == 'VERB':
                    tmp_comment += df_verb.loc[df_verb[(df_verb[1] == domain) & (df_verb['diff']>=0)]['diff'].idxmin()][0] + ' '
                elif word.lemma_ == 'not':
                    continue
                else: 
                    tmp_comment+= word.text + ' '
        else:
            tmp_comment+=(sent.text)
    return tmp_comment

In [20]:
'''
equal to turn_sentences2. Furthermore it adds awesome before every noun, if the sentence shall be positive,
or horrible, if the sentence shall be negative.




turn_sentences3(comment, polarity, domain):
    inputs:
        comment: tokenized original comment
        polarity: predicted polarity (of the comment, preprocessed with clean_comment(text))
        domain: predicted domain
    outputs:
        tmp_comment: pre-processed comment with (hopefully) inverted polatity
'''

def turn_sentences3(comment, polarity, domain):
    tmp_comment = ''
    for sent in comment.sents:
        if polarity == 4:
            for word in sent:
                if word.pos_ == 'ADJ':
                    tmp_comment += df_adj.loc[df_adj[(df_adj[1] == domain)]['diff'].idxmax()][0] + ' '
                elif word.pos_ == 'VERB':
                    tmp_comment += df_verb.loc[df_verb[(df_verb[1] == domain)]['diff'].idxmax()][0] + ' '
                elif word.lemma_ == 'not':
                    continue
                elif word.pos_ == 'NOUN':
                    tmp_comment += ' awesome ' + word.text + ' '
                else: 
                    tmp_comment+= word.text + ' '
        elif polarity == 3:
            for word in sent:
                if word.pos_ == 'ADJ':
                    tmp_comment += df_adj.loc[df_adj[(df_adj[1] == domain) & (df_adj['diff']>=0)]['diff'].idxmin()][0] + ' '
                if word.pos_ == 'VERB':
                    tmp_comment += df_verb.loc[df_verb[(df_verb[1] == domain) & (df_verb['diff']>=0)]['diff'].idxmin()][0] + ' '
                elif word.lemma_ == 'not':
                    continue
                elif word.pos_ == 'NOUN':
                    tmp_comment += ' horrible ' + word.text + ' '
                else: 
                    tmp_comment+= word.text + ' '
        else:
            tmp_comment+=(sent.text)
    return tmp_comment

In [21]:
'''
Master-Loop without original labels:
trys all the described funtion until the comment has the desired polarity. 
If this does not work it adds the original comment.


... und gib's ihm!
'''


new_fakes = []
i=0
for comment, true_domain, true_polarity in zip(df_test[0], df_test[1], df_test[2]):
    polarity = calculate_polarity_i([clean_comment(comment)])[0]
    domain = calculate_domain_i([comment.text])[0]
    tmp_polarity = ''
    tmp_domain = ''
    try_function = 0
    while polarity != tmp_polarity:
        i+=1
        if try_function == 0:
            tmp_comment = assign_nots(comment)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                break
            else:
                try_function+=1
        elif try_function == 1:
            tmp_comment = turn_sentences2(comment, polarity, domain)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                try_function+=1
                break
            else:
                try_function+=1
        elif try_function == 2:
            tmp_comment = turn_sentences(comment, polarity, domain)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                try_function+=1
                break
            else:
                try_function+=1
        elif try_function == 3:
            tmp_comment = turn_sentences3(comment, polarity, domain)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                try_function+=1
                break
            else:
                try_function+=1
        else:
            tmp_polarity = polarity
            print('no success')
            new_fakes.append(['EC10 '+comment.text, true_domain, true_polarity])
            break
        
df_new_fakes = pd.DataFrame(new_fakes)

In [22]:
'''
Run analysis
'''
display(df_new_fakes)
df_new_fakes.to_csv(r'./evaluation_examples.csv', header = None, index=False)
df_new_fakes.to_csv(r'./processed.csv', header = None, index=False)
%run -i "classifier.py"

,0,1,2
0,I have bake this awesome item yet . I bak...,1,4
1,"As far as home not made ice cream not goes ,...",1,3
2,not Great value for your money . Kingston has...,0,3
3,I not purchased a Biopod in March of 2006 . ...,0,4
4,"I have compare this for almost a year , and f...",1,3
...,...,...,...
395,These cables are not super with my not new D...,0,3
396,This does compare the iPAQ hx2755 . Do compa...,0,4
397,The 17 piece set is a not great value but qua...,1,4
398,This product compare compare a excellent day...,0,4


Here is the accuracy for domain: 93.5 %
Here is the accuracy for polarity: 11.25 %


In [23]:
'''
Shows all comments which have the original (undesired) label for polarity
'''

df_new_fakes[3] = calculate_polarity_i(df_new_fakes[0])
df_new_fakes[4] = df_only_text[0]
display(df_new_fakes)
j = 0
for comment, p1, p2, new_comment in zip(df_new_fakes[0], df_new_fakes[2], df_new_fakes[3], df_new_fakes[4]):
    if p1 == p2:
        print(j,': label: ', p1, ' - ', 'new label: ', p2)
        print(comment)
        print()
        print(new_comment)
        print()
        j+=1

,0,1,2,3,4
0,I have bake this awesome item yet . I bak...,1,4,3,I have not received this item yet. I tracked ...
1,"As far as home not made ice cream not goes ,...",1,3,4,"As far as home made ice cream goes, the finnis..."
2,not Great value for your money . Kingston has...,0,3,4,Great value for your money. Kingston has alway...
3,I not purchased a Biopod in March of 2006 . ...,0,4,4,I purchased a Biopod in March of 2006. Receive...
4,"I have compare this for almost a year , and f...",1,3,3,"I have had this for almost a year, and for the..."
...,...,...,...,...,...
395,These cables are not super with my not new D...,0,3,4,These cables are super with my new DVD player....
396,This does compare the iPAQ hx2755 . Do compa...,0,4,3,This does not fit the iPAQ hx2755. Do not try!...
397,The 17 piece set is a not great value but qua...,1,4,4,The 17 piece set is a great value but quality ...
398,This product compare compare a excellent day...,0,4,3,This product stopped working a few days after ...


0 : label:  4  -  new label:  4
I  not purchased a Biopod in March of 2006 .  not Received brand  not new unit that  not included Omnipass software version 2.09 . This software version  not released in 2004.NEWLINE NEWLINE  not Contacted company that  not produced the software and they  not refused to  not upgrade to their  not latest version ( v4 ) that had been  not released a month earlier unless they  not charged me the upgrade fee . NEWLINE NEWLINE The version of software that  not comes with the Biopod ( v2.09 ) pretty much only  not works with Internet Explorer and  not other Microsoft software . The  not new version ( 4 ) supposedly  not works with Firefox and  not several  not other  not non  not -  not Microsoft software programs . NEWLINE NEWLINE I  not recommend you  not avoid this product due to  not old software . I  not returned mine and am  not ordering the IBM Fingerprint Reader 

I purchased a Biopod in March of 2006. Received brand new unit that included Omnipass sof

I couldn't wait to get the Dyson Root 6 - I've had my Dyson DC 07 for 4 years and I LOVE it, so I figured the mini-version would be fantastic. I have 3 Dustbusters (one on each floor) and I was looking forward to the Dyson version, which I was convinced would be top notch.NEWLINE NEWLINE I'm so sorry to be wrong. NEWLINE NEWLINE First off, having to press the button continuously to make it work is a pain. And speaking of pain, the Root is so heavy and unbalanced - all the weight is in the back which puts a lot of strain on your wrist. I haven't noticed any difference in suction to a Dustbuster, but then again, I don't suck up bowling balls. A Dustbuster picks up lint and cat litter just as well.NEWLINE NEWLINE Save your money - this Dyson ain't worth it

7 : label:  4  -  new label:  4
Have you ever  not heard the saying that something  not would  not cost you " an arm and a leg " ? These cables  not cost that , your  not first son , and bidding rights to your estate . Do get me  not w

When  not realtime traffic monitoring  not came to my city , I immediately  not started  not looking at solutions to  not help my wife 's  not daily business driving ( 2 - 500 miles weekly ,  not crisscrossing a 50-mile area through  not major traffic routes ) . I quickly  not settled on an  not automotive GPS with  not realtime traffic monitoring . There are yet  not many choices in this area , so the options  not boiled down to this unit and the Garmin 2720 . ( These  not use FM radio for the traffic info ; the XM - radio units like the TomTom 700 and Garmin 2730 had unacceptably  not high  not monthly costs since I do want XM radio.)NEWLINE NEWLINE I  not chose the Cobra for its  not large screen and  not positive reviews in areas that were  not important to me . Its  not flawed  not routing ability was a  not major issue as it was to be  not used in a well -  not known region . Address lookup and traffic monitoring were the  not primary needs . NEWLINE NEWLINE In  not short , I do 

When realtime traffic monitoring came to my city, I immediately started looking at solutions to help my wife's daily business driving (2-500 miles weekly, crisscrossing a 50-mile area through major traffic routes). I quickly settled on an automotive GPS with realtime traffic monitoring. There are not yet many choices in this area, so the options boiled down to this unit and the Garmin 2720. (These use FM radio for the traffic info; the XM-radio units like the TomTom 700 and Garmin 2730 had unacceptably high monthly costs since I don't want XM radio.)NEWLINE NEWLINE I chose the Cobra for its large screen and positive reviews in areas that were important to me. Its flawed routing ability was not a major issue as it was to be used in a well-known region. Address lookup and traffic monitoring were the primary needs.NEWLINE NEWLINE In short, I don't know much about how the Cobra performs because I was only able to use it for a few short test trips. This is because the mounting hardware is o

...

We  not received the Hamilton Beach BrewStation 12 Cup Coffeemaker as a gift from my sister - in - law .   She  not loved hers and had  not used it for a  not few months so she  not bought one for us .   Shortly after  not buying this for us , her coffeemaker  not started  not leaking .   She  not thought it was just  not worn out from the six months of use and  not purchased a  not different type to  not replace hers . NEWLINE NEWLINE We  not used our BrewStation for the  not first time yesterday .   The night before , I  not set it up and  not programmed it to  not make coffee at 5:00 AM -  not perfect , we  not could get up and have coffee already  not made .   It  not worked great .   A  not minor problem is that I  not enjoy coffee throughout the morning and the coffeemaker  not shut off after two hours .   a  not big deal in the  not grand scheme of things , I just  not turned it back on and life was  not good . NEWLINE NEWLINE Now before I  not go on with my story , I  not need 

It's Logitech...it should have a built in Microphone.  instead there's a small head set.  don't buy this.  spend a little more for a built in microphone

26 : label:  3  -  new label:  3
My excellent purchase of the Q9b  compare in a return as the display was  compare properly within the excellent housing . This  compare in the screen being  compare downwards about 1/4 " and  compare the  compare excellent . The screen also was very excellent even with the brightness  compare up at 100 % .  compare to  compare the unit to  compare it back in the box  compare the base stand to be  compare apart but that was excellent to excellent without  compare it .  compare to  compare , quality control  compare this one . NEWLINE NEWLINE After  compare a excellent Q9b replacement , I unfortunately had a excellent pixel . After  compare the Q9b for a week now , I  compare some excellent  compare between characters . The crispness of the display is excellent but I  compare display fonts to 120dpi to  

My player had the  not same headphones jack problem that so  not many people here have  not had , which  not ended up  not rendering it  not useless ( out of warranty ) .   NEWLINE NEWLINE However , while it  not worked , I was quite  not pleased with the player and the  not included software .   Everything was  not intutive and  not ran smoothly ( except for an  not occational freeze with the player which never  not became  not frequent enough to be a  not major burden ) .   I also  not liked the fact that this player was bound by iTunes or file format restrictions . NEWLINE NEWLINE My  not only  not other complaint was that occasionally the touchpad  not made  not moving  not small increments like up one track or down one track somewhat  not difficult ( just  not give me buttons , I  not know they are sexy but they 're  not easy and  not work well ) .   NEWLINE NEWLINE I am  not giving the product 2 stars because , while I  not liked it , Creative has  not known about the headphones 

(Note to Brian: The WAP54G *can* be configured as a bridge. I have two of them in just such a configuration. Took me about 10 minutes to set them up.)NEWLINE NEWLINE Major modes of operation are:NEWLINE NEWLINE 1. Wireless Access Point (its default set-up)NEWLINE 2. Wireless Client (connect a wired ethernet device via cable to the WAP54G and you now have a wireless ethernet device)NEWLINE 3. Wireless Repeater (extend range of wireless coverage)NEWLINE 4. Wireless Bridge (connect two ethernet networks without running a cable between them -- acts as one end of a "virtual" cable; you need two to get the job done)NEWLINE NEWLINE A pair of wireless bridges is just what I needed but hesitated to buy the WAP54G because of Brian's comment that this wouldn't work. Because of my newness to the wireless world it took a bit of investigation to come to the conclusion that Brian was wrong. I bought a pair and they do work.NEWLINE NEWLINE I rate it 4 instead of 5 stars for a few reasons.NEWLINE NEWLI

This machine is  great looking, compact, and  seems to be the perfect combination fax/answering machine/speakerphone.  One major problem:  The phone is well below average in sound quality,  and the speaker is absolutely terrible.  The tiny, poorly made speaker is placed in the back left side of the unit and and is totaly inaudible at loud volumes due to distortion.  The fax is great,  the print quality is great,  the speed is great, but you have to lower the volume and lean over with your head against the speaker  to hear messages.  This problem also renders the speakerphone useless.  I returned this unit twice thinking it was defective, but spoke to HP and was told that is just the way this unit works.  Buy the Panasonic KX-FHD351  if you  need a great sounding unit.  The only problem with the Panasonic is that it doesn't come in black,  but is a much better phone/answering/speakerphone.

41 : label:  4  -  new label:  4
I did  ask any  horrible change in the  horrible operation of my

Here is the solution with the given (correct) labels:

In [24]:
'''
Master-Loop which uses the labels:
trys all the described funtion until the comment has the desired polarity. 
If this does not work it adds the original comment.


... und gib's ihm!
'''


new_fakes = []
i=0
for comment, true_domain, true_polarity in zip(df_test[0], df_test[1], df_test[2]):
    polarity = true_polarity #calculate_polarity_i([clean_comment(comment)])[0]
    domain = calculate_domain_i([comment.text])[0]
    tmp_polarity = ''
    tmp_domain = ''
    try_function = 0
    while polarity != tmp_polarity:
        i+=1
        if try_function == 0:
            tmp_comment = assign_nots(comment)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                break
            else:
                try_function+=1
        elif try_function == 1:
            tmp_comment = turn_sentences2(comment, polarity, domain)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                try_function+=1
                break
            else:
                try_function+=1
        elif try_function == 2:
            tmp_comment = turn_sentences(comment, polarity, domain)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                try_function+=1
                break
            else:
                try_function+=1
        elif try_function == 3:
            tmp_comment = turn_sentences3(comment, polarity, domain)
            if calculate_polarity_i([tmp_comment])[0] != polarity and calculate_domain_i([tmp_comment])[0] == domain:
                new_fakes.append([tmp_comment, true_domain, true_polarity])
                tmp_polarity == calculate_polarity_i([tmp_comment])[0]
                try_function+=1
                break
            else:
                try_function+=1
        else:
            tmp_polarity = polarity
            print('no success')
            new_fakes.append(['EC10'+comment.text, true_domain, true_polarity])
            break
        
df_new_fakes = pd.DataFrame(new_fakes)

'''
Run analysis
'''
display(df_new_fakes)
df_new_fakes.to_csv(r'./evaluation_examples.csv', header = None, index=False)
%run -i "classifier.py"

,0,1,2
0,I have bake this awesome item yet . I bak...,1,4
1,"As far as home not made ice cream not goes ,...",1,3
2,not Great value for your money . Kingston has...,0,3
3,I compare a Biopod in March of 2006 . compar...,0,4
4,"I have not had this for almost a year , and f...",1,3
...,...,...,...
395,These cables are not super with my not new D...,0,3
396,This does compare the iPAQ hx2755 . Do compa...,0,4
397,The 17 piece set is a perfect value but qualit...,1,4
398,This product compare compare a excellent day...,0,4


Here is the accuracy for domain: 93.5 %
Here is the accuracy for polarity: 0.0 %


In [25]:
#restore original file
df_original.to_csv(r'./evaluation_examples.csv', header = None, index=False)